In [1]:
import json
from utils import statics


In [7]:
# 关于zhidao.dev.json (分析scores和answers)
# answer和fake_answer的match score
data_path = "src/data/dureader/process/2019/v2.0/zhidao.dev.jsonl"
scores = []
with open(data_path, 'r', encoding='utf-8') as f:
    for line in f:
        d = json.loads(line)
        if d["match_score"] > 0:
            scores.append(d["match_score"])
print("about zhidao.train.json match scores: ")
statics(scores, rate=0.90, most_common=5, reversed=True)



about zhidao.train.json match scores: 
total count:  1055
mean: 0.8145590466732651, max: 1.0, min: 0.14285714285714288
most common 5: 
[(1.0, 257), (0.8, 21), (0.923076923076923, 18), (0.6666666666666666, 18), (0.9090909090909091, 15)]
above 0.4 is account for 90.04739336492891%


In [ ]:
# 统计zhidao.test1.json （for only choose one para...）
# 某些sample的documents中document内容可能为空
# 某些sample的paragraph可能为空，但不是全为空
# 如果使用question和para的recall来计算，仍然可能得到score为空(1825/30000多)(启发式：如果score==0， 直接取最前面的那个)
import json
from utils.basic_metric import metric_max_over_ground_truths, recall
zhidao_test_1 = 'src/data/dureader/process/2019/v1.0/zhidao.train.json'
before_para_len = []
best_para_len = []
p_c = 0  # 统计所有para为0的sample个数
with open(zhidao_test_1, 'r', encoding='utf-8') as f:
    for i, line in enumerate(f):
        c = 0  
        sample = json.loads(line)
        # number
        # for doc in sample["documents"]:
        #     # count = len(doc["paragraphs"])
        #     # if count == 0:
        #     #     print(d["question_id"])
        #     for para in doc["paragraphs"]:
        #         c += len(para)
        #         # if len(para) == 0:
        #             # print(d["documents"])
        #             # print(doc["paragraphs"])
        # if c == 0:
        #     p_c += 1
        # ----------------
        # 计算最佳的para
        best_para = []
        # 取最前面三个的不为空的document
        docs = []
        for doc in sample["documents"]:
            if len(doc["segmented_paragraphs"]) != 0:
                docs.append(doc)
        docs = docs[: 3]
        for doc in docs:
            title = doc["segmented_title"]
            # 取前面4个不为空的para
            paras = []
            for para in doc["segmented_paragraphs"]:
                if len(para) != 0:
                    paras.append(para)
            paras = paras[: 4]
            # 全部拼接起来
            best_para = best_para + title
            for para in paras:
                best_para = best_para + para
        before_para_len.append(len(best_para))
        # 截取一定的长度(默认500)
        best_para = best_para[: 500] if len(best_para) > 500 else best_para
        # if len(best_para) == 0:
        #     print(sample["question_id"])
        best_para_len.append(len(best_para))
        if len(best_para) == 0:
            print(sample["question_id"])
# best para len
from utils import statics
print("for choose para len statics: ")
print("before: ")
statics(before_para_len, rate=0.90, most_common=5, reversed=False)
print("after cut: ")
statics(best_para_len, rate=0.90, most_common=5, reversed=False)


In [7]:
# 统计zhidao.test1.json （for  choose multiple paras...）
# 某些sample的documents中document内容可能为空
# 某些sample的paragraph可能为空，但不是全为空
# 如果使用question和para的recall来计算，仍然可能得到score为空(1825/30000多)(启发式：如果score==0， 直接取最前面的那个)
import json
from utils.basic_metric import metric_max_over_ground_truths, recall
zhidao_test_1 = 'src/data/dureader/process/2019/v1.0/zhidao.train.json'
train = True
before_para_len = []
best_para_len = []
p_c = 0  # 统计所有para为0的sample个数
paras_num = []  # 统计paras num分布
with open(zhidao_test_1, 'r', encoding='utf-8') as f:
    for i, line in enumerate(f):
        c = 0  
        sample = json.loads(line)
        # number
        # for doc in sample["documents"]:
        #     # count = len(doc["paragraphs"])
        #     # if count == 0:
        #     #     print(d["question_id"])
        #     for para in doc["paragraphs"]:
        #         c += len(para)
        #         # if len(para) == 0:
        #             # print(d["documents"])
        #             # print(doc["paragraphs"])
        # if c == 0:
        #     p_c += 1
        # ----------------
        # 计算最佳的paras
        best_paras = []
        # 取最前面三个的不为空的document
        docs = []
        if train:
            for doc in sample["documents"]:
                if doc["is_selected"] and len(doc["segmented_paragraphs"]) != 0:
                    docs.append(doc)
        else:
            for doc in sample["documents"]:
                if len(doc["segmented_paragraphs"]) != 0:
                    docs.append(doc)
        docs = docs[: 3]
        for doc in docs:
            c_para = []
            title = doc["segmented_title"]
            # 取前面4个不为空的para
            paras = []
            for para in doc["segmented_paragraphs"]:
                if len(para) != 0:
                    paras.append(para)
            paras = paras[: 4]
            # 将每个doc的tile+4paras拼接
            c_para = c_para + title
            for para in paras:
                c_para = c_para + para
            before_para_len.append(len(c_para))
            # 截取一定的长度(默认500)
            c_para = c_para[: 500] if len(c_para) > 500 else c_para
            best_para_len.append(len(c_para))
            best_paras.append(c_para)
            if len(c_para) == 0:
                print(sample["question_id"])
        paras_num.append(len(best_paras))
# best para len
from utils import statics
print("for choose para len statics: ")
print("before: ")
statics(before_para_len, rate=0.80, most_common=5, reversed=False)
print("after cut: ")
statics(best_para_len, rate=0.90, most_common=5, reversed=False)
print("for paras num:")
statics(paras_num, rate=0.9, most_common=4, reversed=True)


for choose para len statics: 
before: 
total count:  312693
mean: 336.9170496301484, max: 22209, min: 3
most common 5: 
[(40, 1746), (35, 1729), (30, 1729), (39, 1721), (43, 1711)]
below 446 is account for 80.01362358607325%
after cut: 
total count:  312693
mean: 216.46232246964274, max: 500, min: 3
most common 5: 
[(500, 54670), (40, 1746), (35, 1729), (30, 1729), (39, 1721)]
below 500 is account for 100.0%
for paras num:
total count:  135366
mean: 2.3099818270466734, max: 3, min: 0
most common 4: 
[(3, 77754), (2, 27121), (1, 25189), (0, 5302)]
above 1 is account for 96.08321144157321%


In [10]:
# 统计search.test1.json
# 某些sample的documents中document内容可能为空
# 某些sample的paragraph可能为空，但不是全为空
# 如果使用question和para的recall来计算，仍然可能得到score为空(1825/30000多)(启发式：如果score==0， 直接取最前面的那个)
# search.test1.json中有5个sample的para全为空
import json
from utils.basic_metric import metric_max_over_ground_truths, recall
zhidao_test_1 = 'src/data/dureader/process/2019/v1.0/search.test1.json'
before_para_len = []
best_para_len = []
p_c = 0  # 统计所有para为0的sample个数
no_para_samples = []
with open(zhidao_test_1, 'r', encoding='utf-8') as f:
    for i, line in enumerate(f):
        c = 0  
        sample = json.loads(line)
        # number
        for doc in sample["documents"]:
            # count = len(doc["paragraphs"])
            # if count == 0:
            #     print(d["question_id"])
            for para in doc["paragraphs"]:
                c += len(para)
                # if len(para) == 0:
                    # print(d["documents"])
                    # print(doc["paragraphs"])
        if c == 0:
            p_c += 1
        # ----------------
        # 计算最佳的para
        best_para = []
        # 取最前面三个的不为空的document
        docs = []
        for doc in sample["documents"]:
            if len(doc["segmented_paragraphs"]) != 0:
                docs.append(doc)
        docs = docs[: 3]
        for doc in docs:
            title = doc["segmented_title"]
            # 取前面4个不为空的para
            paras = []
            for para in doc["segmented_paragraphs"]:
                if len(para) != 0:
                    paras.append(para)
            paras = paras[: 4]
            # 全部拼接起来
            best_para = best_para + title
            for para in paras:
                best_para = best_para + para
        before_para_len.append(len(best_para))
        # 截取一定的长度(默认500)
        best_para = best_para[: 500] if len(best_para) > 500 else best_para
        # if len(best_para) == 0:
        #     print(sample["question_id"])
        best_para_len.append(len(best_para))
        if len(best_para) == 0:
            data = {}
            print(sample["question_id"])
            data["question_id"] = sample["question_id"]
            data["question"] = sample["question"]
            data["answers"] = []
            data["question_type"] = sample["question_type"]
            data["yesno_answers"] = []
            no_para_samples.append(data)
# write no para samples to file
with open('no_para_search.json', 'w', encoding='utf-8') as f:
    for d in no_para_samples:
        d_str = json.dump(d, f, ensure_ascii=False)
        print("", file=f)
# all len(para) == 0
print("p_c: ", p_c)
# best para len
from utils import statics
print("before: ")
statics(before_para_len, rate=0.98, most_common=5)
print("after cut: ")
statics(best_para_len, rate=0.98, most_common=5)

401621


466589


391912


430591


323995


p_c:  5
before: 
mean: 462.38733333333334, max: 13584, min: 0
most common 5: 
[(330, 81), (243, 81), (263, 78), (208, 76), (303, 75)]
above 98.03666666666668% is 91
after cut: 
mean: 352.15273333333334, max: 500, min: 0
most common 5: 
[(500, 8916), (330, 81), (243, 81), (263, 78), (208, 76)]
above 98.03666666666668% is 91


In [16]:
# 模糊匹配模型路径
import os
import fnmatch
check_point_dir = "src/result/dureader/saved/search/BiDAF"
for file in os.listdir(check_point_dir):
    if fnmatch.fnmatch(file, 'model_best_Rouge_*.pth'):
        os.remove(os.path.join(check_point_dir, file))


In [5]:
from utils import find_zhidao_paras, metric_max_over_ground_truths, recall, f1_score, blue4
import json
import random
# zhidao抽取para分析
data_path = "src/data/dureader/process/2019/v2.0/zhidao.dev.json"
samples = []
with open(data_path, 'r', encoding='utf-8') as f:
    for line in f:
        sample = json.loads(line)
        samples.append(sample)
sample = random.choice(samples)
best_paras = find_zhidao_paras(sample, train=True)
print("question: ", sample["question"])
print("answers: ")
for ans in sample["answers"]:
    print(ans)
print("best_paras:")
for para in best_paras:
    print("".join(para))
    print(len(para))
    score = metric_max_over_ground_truths(recall, para, sample["segmented_question"])
    print("question recall score: ", score)
    score = metric_max_over_ground_truths(recall, para, sample["segmented_answers"])
    print("answers recall score: ", score)

question:  厦门婚纱摄影哪家好xmhhsy
answers: 
花禾摄影。
厦门拾华婚纱摄影。
best_paras:
爱意蔓延_厦门婚纱摄影_厦门婚纱摄影哪家好_厦门摄影工作室_花禾<sep>原创大片玫之私语海花语MR.S&MISS.XMR.S&MISS.XMR.S&MISS.XMR.S&MISS.X<sep>关于花禾花禾婚纱摄影工作室<sep>新品首发蝶恋吾爱MR.S&MISS.X繁花时刻2罗兰陌上花开MR.S&MISS.X<sep>旅拍外景玉女湖爱意蔓延拉市海束河古镇私语天堂角落
97
question recall score:  1.0
answers recall score:  0.75


In [ ]:
# find fake answer span
from utils import find_zhidao_paras, metric_max_over_ground_truths, recall, f1_score, blue4
from utils import find_fake_answer_2, choose_one_para
import json
import random
# zhidao抽取para分析
data_path = "src/data/dureader/process/2019/v2.0/zhidao.dev.json"
samples = []
with open(data_path, 'r', encoding='utf-8') as f:
    for line in f:
        sample = json.loads(line)
        samples.append(sample)
sample = random.choice(samples)
best_paras = find_zhidao_paras(sample, train=True)
if len(best_paras) == 0:
    print("best para len  is 0")
else:
    para = choose_one_para(best_paras, sample["segmented_question"], recall)
    span, s_idx, e_idx, score = find_fake_answer_2(sample, para)
    print("question: ", sample["question"])
    print("gold answers: ", sample["answers"])
    print("*"*100)
    print("span: ", " ".join(span))
    print("s_idx: ", s_idx)
    print("e_idx: ", e_idx)
    print("score: ", score)

question:  治疗鼻炎哪家医院好
gold answers:  ['比较好的医院有：北京同仁医院，北京武警总医院，武警北京市总队第三医院，这些医院治疗鼻炎的效果不错。']
****************************************************************************************************
span:  ， 比较好 的 医院 有 ： 北京同仁医院 ， 北京 武警 总医院 ， 武警 北京市 总队 第 三 医院 ， 这些 医院 治疗 鼻炎 的
s_idx:  99
e_idx:  122
score:  0.9795918367346939


In [55]:
# 保存多个para, 多个找一个answer span
from utils import find_zhidao_paras, metric_max_over_ground_truths, recall, f1_score, blue4
from utils import find_fake_answer_2, choose_one_para, find_fake_answer_from_multi_paras
import json
import random
# zhidao抽取para分析
data_path = "src/data/dureader/process/2019/v2.0/zhidao.dev.json"
samples = []
with open(data_path, 'r', encoding='utf-8') as f:
    for line in f:
        sample = json.loads(line)
        samples.append(sample)
sample = random.choice(samples)
best_paras = find_zhidao_paras(sample, train=True)
if len(best_paras) == 0:
    print("best para len  is 0")
else:
    para = best_paras
    span, s_idx, e_idx, score = find_fake_answer_from_multi_paras(sample, para)
    print("question: ", sample["question"])
    print("gold answers: ")
    for answer in sample["segmented_answers"]:
        print(">>>>>: ", ' '.join(answer))
    print("*"*100)
    print("answer span: ", " ".join(span))
    print("s_idx: ", s_idx)
    print("e_idx: ", e_idx)
    print("score: ", score)
    print("--"*100)
    print("best paras: ")
    for para in best_paras:
        print(">>>", " ".join(para))


question:  慢性支气管炎根治
gold answers: 
>>>>>:  支气管炎 一般 使用 德 辅 的 吣炎清茶 ， 然后 再 配合 食疗 ： ( 1 ) 每日 空腹 吃 糖醋 大蒜 1 ～ 2 根 ， 同时 喝 一些 糖醋 汁 ， 连 服 10 ～ 15 日 。 ( 2 ) 鲜 山药 120 克 ， 去皮 蒸熟 ， 捣 成 泥 状 ， 与 甘蔗汁 200 克 和 匀 再 煮 热 服用 。 1 剂 分 4 次 服 ， 每日 早晚 各 服 1 次 ， 两 日服 完 。 ( 3 ) 烤 熟 去 壳 的 核桃 8 个 ， 压碎 后 与 适量 红糖 拌匀 服用 ， 每日 1 剂 。 用于 哮喘发作 期 。 ( 4 ) 鳖 蛋 3 个 ， 冰糖 20 克 。 将 鳖 蛋 打开 去 壳 ， 加入 少许 黄酒 搅匀 ， 蒸熟 后 调入 冰糖 食 之 ， 每日 服 二 次 。 连 服 3 ～ 4 日 。 对于 有 支气管炎 的 人 。 平时 要 注意 的 是 饮食 以及 其它 的 生活习惯 。 吃 的 方面 宜 吃 青 淡 的 白菜 等 青菜 或 水果 ， 以及 含 蛋白质 较 丰富 的 鸡蛋 ， 牛奶 等 的 食物 ， 少 油腻 。
>>>>>:  支气管炎 一般 使用 调养 配合 改善 饮食 ， 不要 食用 上火 ， 辛辣 食物 ， 可 食 橘红 栀子 | 茶 ， 能 改善 慢性支气管炎 。
>>>>>:  1 、 熟地 20 克 ， 山 萸 12 克 ， 山药 30 克 ， 制 附片 （ 先 熬 ） 20 ~ 30 克 ， 淫羊藿 30 克 ， 核桃仁 30 克 ， 补骨脂 12 克 ， 麻黄 10 克 ， 地龙 12 克 ， 款冬花 12 克 。 水 煎 服 。 2 、 菌 灵芝 30 克 （ 先 熬 ） ， 丹参 30 克 ， 党参 30 克 ， 麦冬 20 克 ， 五味子 20 克 ， 枸杞 20 克 ， 肉桂 6 克 ， 川贝 6 克 ， 甘草 6 克 。 水 煎 服 。 加减 ： 虚寒 者 加 附片 、 熟地 。 痰 热 者 去 肉桂 ， 加 银花 、 桔梗 。 肺燥 者 加 黄精 、 熟地 。 瘀血 症状 明显 者 ， 加 赤芍 、 莪术 。 3 、 蛞蝓 （ 即 无壳 蜗牛 ， 浙江 民间 称为 蜒蚰 螺 ， 四川 民间 称为 漩巴虫 ） ， 洗

In [52]:
# 关于answer span与gold answer的F1值
from utils import f1_score
prediction = ["没有", "这种", "网站", ",", "也", "不可能", "有", "的"]
# 没有 这种 网站 ， 也 不可能 有 的 。
# 没有 这种 网站 , 也 不可能 有 的
ground_truths = ["没有", "这种", "网站", "，", "也", "不可能", "有", "的", "。"]  
f1_score(prediction, ground_truths)


0.823529411764706